3-layer MLP on the CIFAR10 dataset.

Each input image is a 1D vector of size 3072 (=32x32x3).

Implementing the following MLP model:

```
- Input: Bx3072 (B: batch size)
- MLP
  * Layer 1 (linear): #hidden units=512
  * Nonlinearity: nn.ReLU
  * Layer 2 (linear): #hidden units=128
  * Nonlinearity: nn.ReLU  
  * Layer 3 (linear): #hidden units=10 (10 classes)
```

In [1]:
import torch
import torch.nn as nn
# Input: Bx3072 (B: batch size)
# - MLP
  # * Layer 1 (linear): #hidden units=512
  # * Nonlinearity: nn.ReLU
  # * Layer 2 (linear): #hidden units=128
  # * Nonlinearity: nn.ReLU  
  # * Layer 3 (linear): #hidden units=10 (10 classes)
class MLP(nn.Module):
  def __init__(self) -> None:
    super().__init__()
    #### your code starts ####
    self.layer1 = nn.Linear(3072, 512)
    self.relu1 = nn.ReLU()
    self.layer2 = nn.Linear(512, 128)
    self.layer3 = nn.Linear(128, 10)
    #### your code ends ####


  def forward(self, x: torch.Tensor) -> torch.Tensor:
    #### your code starts ####        
    x = self.layer1(x)
    x = self.relu1(x)
    x = self.layer2(x)
    x = self.relu1(x)
    x = self.layer3(x)
    #### your code ends ####
    return x

Computing the cross-entropy loss between prediction and gt.

In [2]:
import torch
import torch.nn as nn

# unit-test code
batch_size = 5
num_class = 10


gt = torch.randint(0, num_class, (batch_size,))


pred = torch.rand((batch_size, num_class)) * 1234


# loss function
loss_fn = nn.CrossEntropyLoss()
loss_unit_test = loss_fn(pred, gt)

print(loss_unit_test)

tensor(606.3922)


Optimazation: SGD with `lr_rate = 0.01`.

Creating a model from the MLP class
Creating SGD optimizer upon this model's parameters

In [3]:
model = MLP()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

Dataset: [CIFAR10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html). Each image is of the size 32x32x3, containing one main object out of 10 categories.

<img height=400 src="http://bc-cv.github.io/csci3343/public/cifar10/cifar10.png"/>

**Background:**

For SGD, we need to create a dataset class and a dataloader class to randomly sample batches of data (image, label) to train the model. Following the example here [link](https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel). 

Downloading CIFAR10 data
Creating Torch tensors `X_train` and `Y_train` using data_batch_1; `X_test` and `Y_test` using data_batch_2. Note that `X` is a Nx3072 matrix, where each image is flattened.

In [4]:
! wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O cifar-10-python.tar.gz
! tar -xzvf cifar-10-python.tar.gz

--2022-10-17 21:58:05--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  27.7MB/s    in 6.1s    

2022-10-17 21:58:13 (26.5 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

x cifar-10-batches-py/
x cifar-10-batches-py/data_batch_4
x cifar-10-batches-py/readme.html
x cifar-10-batches-py/test_batch
x cifar-10-batches-py/data_batch_3
x cifar-10-batches-py/batches.meta
x cifar-10-batches-py/data_batch_2
x cifar-10-batches-py/data_batch_5
x cifar-10-batches-py/data_batch_1


In [5]:
import numpy as np
import matplotlib.pyplot as plt

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# read in batch-1 as training data
data_b1 = unpickle('cifar-10-batches-py/data_batch_1')
# each image is a 1D vector
train_im1d = data_b1[b'data']
train_im = train_im1d.reshape([train_im1d.shape[0],3,32,32]).transpose([0,2,3,1])
train_label = data_b1[b'labels']

# read in batch-2 as validation data
data_b2 = unpickle('cifar-10-batches-py/data_batch_2')
val_im1d = data_b2[b'data']
val_im = val_im1d.reshape([10000,3,32,32]).transpose([0,2,3,1])
val_label = data_b2[b'labels']

label_name = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [6]:
X_train = torch.from_numpy(train_im1d).float()
Y_train = torch.from_numpy(np.array(train_label)).long()


X_test = torch.from_numpy(val_im1d).float()
Y_test = torch.from_numpy(np.array(val_label)).long()

  -  `__init__()`: makes attribute variables to record the input
  -  `__len__()`: returns the number of images in the dataset
  -  `__getitem__()`: Pytorch dataloader calls this function with an input random index `index` between `[0, len-1]` and ask for the return of `index`-th image and its label. The dataloader will later make them into the tensor format for each batch during each training iteration.

In [7]:
import torch
class CIFAR10Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, X, Y):
    self.X = X/255
    self.Y = Y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.Y[index]

In [8]:
train_dataset = CIFAR10Dataset(X_train, Y_train)
test_dataset = CIFAR10Dataset(X_test, Y_test)

- creating the `train_dataloader` and `test_dataloader` for each dataset with the following parameters:
```
batch_size: 64,
shuffle: True,
num_workers': 1
```

In [9]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

# <b>Train and test</b>

Train the model for 200 epochs.

In [10]:
# put model on GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)


# training loop
num_epochs = 200
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_dataset)//64, loss.item()))


Epoch [1/200], Step [100/156], Loss: 2.2769
Epoch [2/200], Step [100/156], Loss: 2.1854
Epoch [3/200], Step [100/156], Loss: 1.9706
Epoch [4/200], Step [100/156], Loss: 1.9789
Epoch [5/200], Step [100/156], Loss: 1.9221
Epoch [6/200], Step [100/156], Loss: 1.9348
Epoch [7/200], Step [100/156], Loss: 1.9082
Epoch [8/200], Step [100/156], Loss: 1.7156
Epoch [9/200], Step [100/156], Loss: 1.8115
Epoch [10/200], Step [100/156], Loss: 1.9201
Epoch [11/200], Step [100/156], Loss: 1.8768
Epoch [12/200], Step [100/156], Loss: 1.6641
Epoch [13/200], Step [100/156], Loss: 1.7824
Epoch [14/200], Step [100/156], Loss: 1.8132
Epoch [15/200], Step [100/156], Loss: 1.6293
Epoch [16/200], Step [100/156], Loss: 1.6594
Epoch [17/200], Step [100/156], Loss: 1.6600
Epoch [18/200], Step [100/156], Loss: 1.7549
Epoch [19/200], Step [100/156], Loss: 1.6597
Epoch [20/200], Step [100/156], Loss: 1.7039
Epoch [21/200], Step [100/156], Loss: 1.6167
Epoch [22/200], Step [100/156], Loss: 1.5836
Epoch [23/200], Ste

In [11]:
# testing loop
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        correct += (labels.data == outputs.argmax(axis=1)).sum()
        total += labels.size(0)
    accuracy = 100 * correct / total
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(accuracy))

Test Accuracy of the model on the 10000 test images: 41.27000045776367 %
